In [2]:
import numpy as np
import csv
from collections import defaultdict

truth_data = defaultdict(dict)
train_data = defaultdict(dict)

with open("../data/test2_new.txt", 'r') as fTruth:
    next(fTruth)  # Skip the header row
    for line in fTruth:
        trainUserID, trainItemID, trainRating = line.strip().split('|')
        truth_data[trainUserID][trainItemID] = int(trainRating)

with open("../data/trainIdx2_matrix.txt", 'r') as fTrain:
    next(fTrain)  # Skip the header row
    for line in fTrain:
        trainUserID, trainItemID, trainRating = line.strip().split('|')
        train_data[trainUserID][trainItemID] = int(trainRating)

track_rating_data = defaultdict(list)

for user_ratings in train_data.values():
    for trackID, rating in user_ratings.items():
        track_rating_data[trackID].append(rating)

track_stats = {}
for trackID, ratings in track_rating_data.items():
    ratings_array = np.array(ratings)
    track_stats[trackID] = {
        'min': np.min(ratings_array),
        'max': np.max(ratings_array),
        'mean': np.mean(ratings_array),
        'variance': np.var(ratings_array),
        'median': np.median(ratings_array)
    }

with open("../data/testTrack_hierarchy.txt", 'r') as fTest, open("../data/proccessed/new.csv", 'w', newline='') as fOut:
    csv_writer = csv.writer(fOut)
    csv_writer.writerow(["UserId", "TrackId", "AlbumId", "ArtistId", "AlbumRating", "ArtistRating", "TotalScore", "Predictor",
                         "MinRating", "MaxRating", "MeanRating", "Variance", "MedianRating"])

    trackID_vec = [0] * 6
    albumID_vec = [0] * 6
    artistID_vec = [0] * 6
    lastUserID = -1
    total_sum = 0
    total_count = 0

    for line in fTest:
        userID, trackID, albumID, artistID, *_ = line.strip().split('|')

        if userID != lastUserID:
            ii = 0
            user_rating_inTrain = np.zeros(shape=(6, 2))

        trackID_vec[ii] = trackID
        albumID_vec[ii] = albumID
        artistID_vec[ii] = artistID
        ii += 1
        lastUserID = userID

        if userID not in truth_data:
            continue

        if ii == 6:
            user_train_data = train_data[userID]

            for nn in range(6):
                user_rating_inTrain[nn] = [
                                           user_train_data.get(
                                               albumID_vec[nn], 0),
                                           user_train_data.get(artistID_vec[nn], 0)]
            for nn in range(6):
                total_score = sum(user_rating_inTrain[nn])
                total_sum += total_score
                total_count += 1
                prediction = truth_data[userID][trackID_vec[nn]]
                track_stat = track_stats.get(trackID_vec[nn], {})

                csv_writer.writerow([userID, trackID_vec[nn], albumID_vec[nn],
                                     artistID_vec[nn], *
                                     user_rating_inTrain[nn], total_score, prediction,
                                     track_stat.get(
                                         'min', 0), track_stat.get('max', 0),
                                     track_stat.get('mean', 0), track_stat.get(
                                         'variance', 0),
                                     track_stat.get('median', 0)])

print("Total:" + str(total_sum))
print("Count: " + str(total_count))
print("Average: " + str(total_sum/total_count))


Total:352816.0
Count: 6000
Average: 58.80266666666667


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, col
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml import Pipeline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import roc_curve, auc
from pyspark.sql.functions import split, col, avg


spark = SparkSession.builder.appName('recommend-ML').getOrCreate()
# import test_2 ground truths
data_dir = "../data/proccessed/new.csv"
df = spark.read.csv(data_dir, header=True, inferSchema=True)
df.show(5)




your 131072x1 screen size is bogus. expect trouble
23/05/08 21:33:05 WARN Utils: Your hostname, BryanDesktop resolves to a loopback address: 127.0.1.1; using 172.23.1.37 instead (on interface eth0)
23/05/08 21:33:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/08 21:33:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/08 21:33:07 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/05/08 21:33:07 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


+------+-------+-------+--------+-----------+------------+----------+---------+---------+---------+-----------------+------------------+------------+
|UserId|TrackId|AlbumId|ArtistId|AlbumRating|ArtistRating|TotalScore|Predictor|MinRating|MaxRating|       MeanRating|          Variance|MedianRating|
+------+-------+-------+--------+-----------+------------+----------+---------+---------+---------+-----------------+------------------+------------+
|200031|  30877| 192723|  132319|       90.0|        50.0|     140.0|        1|        0|      100|53.42664670658683|1195.6428228871598|        60.0|
|200031|   8244| 223220|  233697|       90.0|         0.0|      90.0|        1|        0|      100|55.75752773375594|1216.3168014948726|        70.0|
|200031| 130183|   None|    None|        0.0|         0.0|       0.0|        0|        0|       90|57.69230769230769| 1063.905325443787|        70.0|
|200031| 198762| 220103|  113265|        0.0|         0.0|       0.0|        0|        0|      100|6